<a href="https://colab.research.google.com/github/ghedin-alison/NLP/blob/main/Template_Trabalho_Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Participantes (RM - NOME):***<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de chamados aplicando técnicas de PLN**
---

A **DinDinAgora** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

Obs.: Para a métrica F1 Score, usar o parâmetro average = 'weighted'.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

In [1]:
# instalações de pacotes
!pip install spacy
!python -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-22 00:38:31.169796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 00:38:32.958311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-22 00:38:32.958454: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nv

In [2]:
#import de bibliotecas
import spacy
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

import re
import spacy
import nltk



In [16]:
# CARREGANDO O DATA FRAME
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

In [75]:
df.head(20)

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,text_treat
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",bom dia meu nome é e agradeço se você puder me...
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,atualizei meu cartão em e fui informado pelo ...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...,o cartão chase foi relatado em no entanto o p...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti...",em enquanto tentava reservar um ticket me dep...
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",meu neto me dê cheque por eu depositei o na ...
5,3209411,2019-04-11T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Você pode remover a consulta,você pode remover a consulta
6,3331023,2019-08-06T12:00:00-05:00,Serviços de conta bancária,Sem aviso prévio J.P. Morgan Chase restringiu ...,sem aviso prévio jp morgan chase restringiu mi...
7,3352857,2019-08-24T12:00:00-05:00,Outros,"Durante os meses de verão, experimento uma ren...",durante os meses de verão eperimento uma renda...
8,3226110,2019-04-29T12:00:00-05:00,Roubo / Relatório de disputa,"Em xxxx xx/xx/2019, fiz um pagamento {$ 300.00...",em fiz um pagamento a um varejista on line ...
9,3237765,2019-05-09T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Eu tenho um cartão de crédito Chase que está r...,eu tenho um cartão de crédito chase que está r...


In [77]:
df.descricao_reclamacao[8]

'Em xxxx xx/xx/2019, fiz um pagamento {$ 300.00} a um varejista on -line usando o Chase Quick Pay com xxxx no site da Chase. Percebi que este era um site de fraudes após nenhuma confirmação, produto ou resposta a quaisquer perguntas sobre o pedido e contatou o Chase para obter mais informações sobre a transação para tentar obter um reembolso pelo banco de varejistas.\r\n\r\nEntrei em contato com o Chase por meio de uma mensagem segura em xx/xx/2019 explicando o que aconteceu e perguntei: "Existe uma maneira de reverter essa transação, ou você tem um contato no XXXX que pode me fornecer mais informações sobre o destinatário? \'\' Que xxxx , minha mensagem foi revisada e recebi uma resposta reafirmando meu e -mail original e me informando que "a transferência foi concluída. No entanto, como você mencionou que o site é uma farsa enquanto podemos lidar com a maioria das perguntas por e -mail, algumas exigem a experiência de outra equipe. Para ajudá -lo melhor em relação ao reembolso, solic

In [78]:
df.descricao_reclamacao[10]

'Mishandling desta conta por Chase Auto e XXXX.'

In [19]:
df.shape

(21072, 4)

In [20]:
# aqui é melhor equilibrar a base de dados
df.categoria.value_counts()

Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: categoria, dtype: int64

In [21]:
nlp = spacy.load("pt_core_news_sm")

In [22]:
#remover o XXXX
def limpar_textos(texto):
  texto = re.sub(r'[,.:;]', '', texto)
  texto = re.sub(r'\n', ' ', texto)
  texto = re.sub(r'[xxxx]', '', texto)
  texto = re.sub('\\s+', ' ', texto)
  texto = re.sub(u'[^a-zA-ZéúíóáÉÚÍÓÁèùìòàÈÙÌÒÀõãñÕÃÑêûîôâÊÛÎÔÂëÿüïöäËYÜÏÖÄçÇ ]', '', texto)
  texto = texto.strip().lower()
  return texto

In [23]:
# função de lematização completa do documento
def lematiza_texto(texto: str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    sent.append(word.lemma_)
  return " ".join(sent)

# função de lematização de verbos do documento
def lematiza_verbos(texto:str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    if word.pos_ == "VERB":
      sent.append(word.lemma_)
    else:
      sent.append(word.text)
  return " ".join(sent)


In [24]:
df["text_treat"] = df.descricao_reclamacao.apply(limpar_textos)

In [26]:
df.text_treat.head()

0    bom dia meu nome é e agradeço se você puder me...
1    atualizei meu cartão em  e fui informado pelo ...
2    o cartão chase foi relatado em  no entanto o p...
3    em  enquanto tentava reservar um ticket me dep...
4    meu neto me dê cheque por   eu depositei o na ...
Name: text_treat, dtype: object

In [27]:
# divisão das amostras em treino e teste
df_raw, df_amostra = train_test_split(df, 
                                       test_size=0.1, 
                                       random_state=42)

In [28]:
# aplica a lematização de no dataframe criando novas colunas
df_amostra["text_lemma"] = df_amostra.text_treat.apply(lematiza_texto)

In [29]:
# aplica a lematização de no dataframe criando novas colunas
df_amostra["text_lemma_verbos"] = df_amostra.text_treat.apply(lematiza_verbos)

In [30]:
# divisão das amostras em treino e teste
df_treino, df_teste = train_test_split(df_amostra, 
                                       test_size=0.3, 
                                       random_state=42)

In [31]:
# Regressão Logistica com lematização em verbos

nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

#stopwords combinadas
stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))

# vetorização com Tfid
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [34]:
f1_score(df_teste['categoria'], pred, average='weighted')

0.8985692993239285

In [33]:
from nltk.stem.snowball import stopwords
#Contagem de termos
vect = CountVectorizer(ngram_range=(1,1))
vect = CountVectorizer(ngram_range=(1,2))
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_spacy)
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops_nltk)


# Frequencia de Termos(TF)
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, norm='l1')
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False)
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops)
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops_nltk)

# Frequencia de termos invertida
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1')
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, norm='l1')
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, norm='l2')
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, norm='l1', stop_words=stops)
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, norm='l2', stop_words=stops_spacy)





In [36]:
# Teste 2
# vetorização com Tfid
vetor_tfid = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=list(stops_spacy), norm='l1')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.7802398306030581

In [39]:
# Teste 3
# vetorização com Tfid
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=False, stop_words=stops_nltk, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.8782722698021378

In [40]:
# Teste 4
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.866905015625206

In [42]:
# Teste 5
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.855687415083321

In [43]:
# Teste 6
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

#treina modelo
tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = tree_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.6454306440697523

In [44]:
# Teste 7
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df_treino.text_lemma_verbos)
text_vect_treino = vect.transform(df_treino.text_lemma_verbos)

#treina modelo
tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma_verbos)
pred = tree_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.6404016210671335

In [45]:
# Teste 8
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

#treina modelo
tree_model = DecisionTreeClassifier()
tree_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma)
pred = tree_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.6687604147553189

In [46]:
# 9
# vetorização com Tfid
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

#treina modelo
trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma)
pred = trees_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.7901213938656896

In [47]:
# 10
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_treino.text_lemma)
text_vect_treino = vect.transform(df_treino.text_lemma)

#treina modelo
trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vect.transform(df_teste.text_lemma)
pred = trees_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.7749431398039041

In [53]:
# 11
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')

vetor_tfid.fit(df_treino.text_lemma_verbos)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma_verbos)

#treina modelo
trees_model = RandomForestClassifier()
trees_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma_verbos)
pred = trees_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])
f1_score(df_teste['categoria'], pred, average='weighted')

0.8120579923615779

In [84]:
# testando mais algumas variações no modelo campeão até agora..mínimo 90 pra bater

# Regressão Logistica com lematização em verbos
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

#stopwords combinadas
stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))

# vetorização com Tfid

vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
f1_score(df_teste['categoria'], pred, average='weighted')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.8985692993239285

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.